#Library Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.stats import entropy
import math
from functools import reduce
from sklearn.feature_selection import mutual_info_classif


#Data Reading

In [ ]:
data = pd.read_csv('EWSGSE967.txt')
data.head()

,31307_at,31309_r_at,31313_at,31314_at,31315_at,31316_at,31317_r_at,31320_at,31321_at,31322_at,31323_r_at,31324_at,31328_at,31329_at,31330_at,31331_at,31333_at,31335_at,31339_at,31344_at,31345_at,31349_at,31350_at,31352_at,31353_f_at,31356_at,31357_at,31359_at,31363_at,31364_i_at,31365_f_at,31371_at,31372_at,31373_at,31376_at,31378_at,31379_at,31383_at,31384_at,31385_at,...,142_at,146_at,147_at,148_at,149_at,150_at,151_s_at,153_f_at,154_at,113_i_at,114_r_at,116_at,117_at,119_at,120_at,121_at,124_i_at,125_r_at,126_s_at,127_at,128_at,129_g_at,131_at,132_at,133_at,134_at,135_g_at,111_at,100_g_at,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,class,sample
0,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,d,string
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class,meta
2,10,10,206.7,217.3,95,1590.1,10,1112.9,118.2,10,725.7,10,62,92590.8,10,254.8,264.2,10,10,482.7,151,131,10,1795.2,10,145.2,176.8,10,10,342.4,10,1204.9,10,10.8,86.1,53.7,53.3,10,35709,764.6,...,570.5,1743.5,908.1,1694.4,124.3,100697.8,10,10,10,10,530.3,59.7,10,657.7,4162.8,10,10,58.2,10,830.6,10,10,10,416.7,10,10,1296.7,387.6,10,226.2,414.8,672.9,10,5141.4,57.9,10,2595.3,1340.2,EWS,EWS
3,10,10,10,10,289.1,10,2350.7,207.4,456.3,402.9,10,412.8,386.2,10,157825.8,10,348.1,294.3,10,610.6,86.8,10,10,243.4,897.3,581.1,310.1,10,21.3,10,168.2,10,891.6,10,29.4,10,343,10,10,115624.3,...,10,10,1918.6,1007.7,1991.1,275.2,58563.3,374.4,10,447.5,10,493.6,10,10,1685.9,5417.5,10,10,73.1,10,16.5,10,611.8,10,802.9,626.6,230.7,1247.7,950.5,1597.2,315.3,1458,10,530.7,1618.1,55.5,567.4,2517.4,EWS,EWS
4,10,10,10,10,606.2,316.1,1061.9,10,488.2,98.1,110,10,260.2,322.9,110835.7,34.2,10,10,198.8,820.8,275.4,260.2,10,425.5,964.7,646.7,588.2,10,204.2,10,333.1,10,246.4,206.2,418.3,10,104.5,10,10,61946.8,...,10,10,1392.3,676.9,644.9,702.8,68256.4,318.1,51.6,134.4,332.2,160.7,10,10,951.4,4231.3,10,10,454.5,10,2407.7,1212,934.6,10,1126.3,119.7,10,748,649.1,318.6,814.4,1189.7,97.9,431.2,2856.7,10,10,3572.6,EWS,EWS


In [ ]:
data = data.drop(data.index[[0,1]])
data = data.drop(['sample'], axis=1)
data.index=range(0,len(data),1)
data.head()

,31307_at,31309_r_at,31313_at,31314_at,31315_at,31316_at,31317_r_at,31320_at,31321_at,31322_at,31323_r_at,31324_at,31328_at,31329_at,31330_at,31331_at,31333_at,31335_at,31339_at,31344_at,31345_at,31349_at,31350_at,31352_at,31353_f_at,31356_at,31357_at,31359_at,31363_at,31364_i_at,31365_f_at,31371_at,31372_at,31373_at,31376_at,31378_at,31379_at,31383_at,31384_at,31385_at,...,141_s_at,142_at,146_at,147_at,148_at,149_at,150_at,151_s_at,153_f_at,154_at,113_i_at,114_r_at,116_at,117_at,119_at,120_at,121_at,124_i_at,125_r_at,126_s_at,127_at,128_at,129_g_at,131_at,132_at,133_at,134_at,135_g_at,111_at,100_g_at,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,class
0,10,10,206.7,217.3,95,1590.1,10,1112.9,118.2,10,725.7,10,62,92590.8,10,254.8,264.2,10,10,482.7,151,131,10,1795.2,10,145.2,176.8,10,10,342.4,10,1204.9,10,10.8,86.1,53.7,53.3,10,35709,764.6,...,10,570.5,1743.5,908.1,1694.4,124.3,100697.8,10,10,10,10,530.3,59.7,10,657.7,4162.8,10,10,58.2,10,830.6,10,10,10,416.7,10,10,1296.7,387.6,10,226.2,414.8,672.9,10,5141.4,57.9,10,2595.3,1340.2,EWS
1,10,10,10,10,289.1,10,2350.7,207.4,456.3,402.9,10,412.8,386.2,10,157825.8,10,348.1,294.3,10,610.6,86.8,10,10,243.4,897.3,581.1,310.1,10,21.3,10,168.2,10,891.6,10,29.4,10,343,10,10,115624.3,...,10,10,10,1918.6,1007.7,1991.1,275.2,58563.3,374.4,10,447.5,10,493.6,10,10,1685.9,5417.5,10,10,73.1,10,16.5,10,611.8,10,802.9,626.6,230.7,1247.7,950.5,1597.2,315.3,1458,10,530.7,1618.1,55.5,567.4,2517.4,EWS
2,10,10,10,10,606.2,316.1,1061.9,10,488.2,98.1,110,10,260.2,322.9,110835.7,34.2,10,10,198.8,820.8,275.4,260.2,10,425.5,964.7,646.7,588.2,10,204.2,10,333.1,10,246.4,206.2,418.3,10,104.5,10,10,61946.8,...,10,10,10,1392.3,676.9,644.9,702.8,68256.4,318.1,51.6,134.4,332.2,160.7,10,10,951.4,4231.3,10,10,454.5,10,2407.7,1212,934.6,10,1126.3,119.7,10,748,649.1,318.6,814.4,1189.7,97.9,431.2,2856.7,10,10,3572.6,EWS
3,10,124,10,174.7,331.8,253.3,1227.3,372.7,462.1,10,10,597.3,292.5,119.3,114888.1,206.2,10,10,10,10,25.7,10,10,10,1449.3,431.8,336.7,160.8,328.8,234.8,380.5,316.5,1089.8,10,25.1,147.9,307.9,46.4,10,56772.1,...,10,10,163.8,1624.8,955.6,1106.2,81.6,116916,482.9,286.4,10,203.7,47.6,337.1,10,617.7,5301.4,10,10,329.3,10,10,234.5,10,26.4,320.4,10,235.5,687.7,1025.2,734.5,10,1692,338.4,10,7958.5,481.1,10,2695.1,EWS
4,10,10,10,74.9,10,10,1719.8,10,748.5,245.7,140.6,1111.1,909.5,602.9,99968.9,149.5,542.6,10,10,592.3,390.8,10,10,10,1066.6,654.2,10,319.6,78.4,267.3,475.5,10,1592.5,252.3,10,10,617.6,10,10,12625.8,...,1915.1,10,10,2189.1,617.5,10,10,51164.2,10,10,664.1,10,23.4,56.2,10,840.1,5000.2,10,88.6,205.3,10,322.5,10,204.5,145.4,1042.7,32.1,10,10,535.2,10,403.5,2527.6,518.1,10,1772.2,933.2,10,1754.4,EWS


In [ ]:
data.dropna()
data.isnull().sum()

31307_at      0
31309_r_at    0
31313_at      0
31314_at      0
31315_at      0
             ..
106_at        0
107_at        0
108_g_at      0
109_at        0
class         0
Length: 9946, dtype: int64

In [ ]:
dict = {
    'class':{
        'EWS':0,
        'RMS':1
    }
}   
data=data.replace(dict)
data.head()


,31307_at,31309_r_at,31313_at,31314_at,31315_at,31316_at,31317_r_at,31320_at,31321_at,31322_at,31323_r_at,31324_at,31328_at,31329_at,31330_at,31331_at,31333_at,31335_at,31339_at,31344_at,31345_at,31349_at,31350_at,31352_at,31353_f_at,31356_at,31357_at,31359_at,31363_at,31364_i_at,31365_f_at,31371_at,31372_at,31373_at,31376_at,31378_at,31379_at,31383_at,31384_at,31385_at,...,141_s_at,142_at,146_at,147_at,148_at,149_at,150_at,151_s_at,153_f_at,154_at,113_i_at,114_r_at,116_at,117_at,119_at,120_at,121_at,124_i_at,125_r_at,126_s_at,127_at,128_at,129_g_at,131_at,132_at,133_at,134_at,135_g_at,111_at,100_g_at,101_at,102_at,103_at,104_at,105_at,106_at,107_at,108_g_at,109_at,class
0,10,10,206.7,217.3,95,1590.1,10,1112.9,118.2,10,725.7,10,62,92590.8,10,254.8,264.2,10,10,482.7,151,131,10,1795.2,10,145.2,176.8,10,10,342.4,10,1204.9,10,10.8,86.1,53.7,53.3,10,35709,764.6,...,10,570.5,1743.5,908.1,1694.4,124.3,100697.8,10,10,10,10,530.3,59.7,10,657.7,4162.8,10,10,58.2,10,830.6,10,10,10,416.7,10,10,1296.7,387.6,10,226.2,414.8,672.9,10,5141.4,57.9,10,2595.3,1340.2,0
1,10,10,10,10,289.1,10,2350.7,207.4,456.3,402.9,10,412.8,386.2,10,157825.8,10,348.1,294.3,10,610.6,86.8,10,10,243.4,897.3,581.1,310.1,10,21.3,10,168.2,10,891.6,10,29.4,10,343,10,10,115624.3,...,10,10,10,1918.6,1007.7,1991.1,275.2,58563.3,374.4,10,447.5,10,493.6,10,10,1685.9,5417.5,10,10,73.1,10,16.5,10,611.8,10,802.9,626.6,230.7,1247.7,950.5,1597.2,315.3,1458,10,530.7,1618.1,55.5,567.4,2517.4,0
2,10,10,10,10,606.2,316.1,1061.9,10,488.2,98.1,110,10,260.2,322.9,110835.7,34.2,10,10,198.8,820.8,275.4,260.2,10,425.5,964.7,646.7,588.2,10,204.2,10,333.1,10,246.4,206.2,418.3,10,104.5,10,10,61946.8,...,10,10,10,1392.3,676.9,644.9,702.8,68256.4,318.1,51.6,134.4,332.2,160.7,10,10,951.4,4231.3,10,10,454.5,10,2407.7,1212,934.6,10,1126.3,119.7,10,748,649.1,318.6,814.4,1189.7,97.9,431.2,2856.7,10,10,3572.6,0
3,10,124,10,174.7,331.8,253.3,1227.3,372.7,462.1,10,10,597.3,292.5,119.3,114888.1,206.2,10,10,10,10,25.7,10,10,10,1449.3,431.8,336.7,160.8,328.8,234.8,380.5,316.5,1089.8,10,25.1,147.9,307.9,46.4,10,56772.1,...,10,10,163.8,1624.8,955.6,1106.2,81.6,116916,482.9,286.4,10,203.7,47.6,337.1,10,617.7,5301.4,10,10,329.3,10,10,234.5,10,26.4,320.4,10,235.5,687.7,1025.2,734.5,10,1692,338.4,10,7958.5,481.1,10,2695.1,0
4,10,10,10,74.9,10,10,1719.8,10,748.5,245.7,140.6,1111.1,909.5,602.9,99968.9,149.5,542.6,10,10,592.3,390.8,10,10,10,1066.6,654.2,10,319.6,78.4,267.3,475.5,10,1592.5,252.3,10,10,617.6,10,10,12625.8,...,1915.1,10,10,2189.1,617.5,10,10,51164.2,10,10,664.1,10,23.4,56.2,10,840.1,5000.2,10,88.6,205.3,10,322.5,10,204.5,145.4,1042.7,32.1,10,10,535.2,10,403.5,2527.6,518.1,10,1772.2,933.2,10,1754.4,0


#Data Split

In [178]:
X = data.drop(['class'], axis=1)
y = data['class']
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.6)
print(len(x_train),len(x_test),len(y_train),len(y_test))

9 14 9 14


#Filter Techniques

*Correlation*

In [ ]:
df1 = X
op=y
a1=[]
c1=[]
for i in df1.columns.values:
        df1[i]=df1[i].astype('float64')
        correlation=op.corr(df1[i])
        if correlation>0.2:
            a1.append(correlation)
            c1.append(i)

dfc1=pd.DataFrame(c1)
dfa1=pd.DataFrame(a1)
info1 = pd.concat([dfc1,dfa1], axis=1)
info1.columns = ['name','correlation']
info1

,name,correlation
0,31307_at,0.245531
1,31309_r_at,0.328682
2,31317_r_at,0.431369
3,31320_at,0.254630
4,31321_at,0.328553
...,...,...
4414,100_g_at,0.315135
4415,103_at,0.420099
4416,104_at,0.326254
4417,107_at,0.365811


*Selecting top 50*

In [ ]:
top1 = info1.nlargest(50,'correlation')
SelectedFeatures1= np.array(top1['name'])
SelectedFeatures1

array(['40570_at', '35321_at', '35330_at', '39350_at', '39690_at',
       '38486_at', '36943_r_at', '1664_at', '36308_at', '39542_at',
       '39063_at', '40004_at', '649_s_at', '37749_at', '36118_at',
       '33192_g_at', '41289_at', '1296_at', '41470_at', '35312_at',
       '37605_at', '38031_at', '820_at', '34408_at', '33911_at',
       '33405_at', '36113_s_at', '33929_at', '38713_at', '39327_at',
       '2057_g_at', '659_g_at', '35824_at', '35648_at', '652_g_at',
       '404_at', '35832_at', '35850_at', '32845_at', '41686_s_at',
       '38809_s_at', '39084_at', '37735_at', '484_at', '33404_at',
       '260_at', '36596_r_at', '36114_r_at', '37210_at', '41836_at'],
      dtype=object)

*Info Gain Ratio*

In [194]:
from scipy.stats import entropy
def info_gain(Ex,a):
    H_Ex = entropy(list(Counter(Ex).values()))
    # Compute the sum of all values v in a
    sum_v = 0
    for v in set(a):
        Ex_a_v = [x for x, t in zip(Ex, a) if t == v]
        sum_v += (len(Ex_a_v) / len(Ex)) *\
                 (entropy(list(Counter(Ex_a_v).values())))

    # Return result
    return H_Ex - sum_v


def intrinsic_value(Ex, a):
    # Compute the sum of all values v in a
    sum_v = 0
    for v in set(a):
        Ex_a_v = [x for x, t in zip(Ex, a) if t == v]
        sum_v += (len(Ex_a_v) / len(Ex)) * math.log(len(Ex_a_v) / len(Ex), 2)

    # Return result
    return -sum_v


def info_gain_ratio(Ex, a):
        # Check whether examples and attributes have the same lengths.
    if len(Ex) != len(a):
        raise ValueError("Ex and a must be of the same size.")

    # Compute information gain ratio as IG/IV
    return info_gain(Ex, a) / intrinsic_value(Ex, a)


In [195]:
ar=[]
br=[]
iv=[]
ig=[]
for (columnName, columnData) in X.iteritems():
        a=info_gain_ratio(columnData, y)
        c=intrinsic_value(columnData, y)
        d=info_gain(columnData, y)
        ar.append(a)
        br.append(columnName)
        iv.append(c)
        ig.append(d)
a1 = pd.DataFrame(ar)
b1 = pd.DataFrame(br)
info2 = pd.concat([a1,b1], axis=1)
info2.columns = ['Score','Features']
info2

,Score,Features
0,0.253627,31307_at
1,0.379228,31309_r_at
2,0.120911,31313_at
3,0.423969,31314_at
4,0.379228,31315_at
...,...,...
9940,0.302509,105_at
9941,0.693147,106_at
9942,0.510816,107_at
9943,0.215756,108_g_at


In [ ]:
top2 = info2.nlargest(50,'Score')
SelectedFeatures2= np.array(top2['Features'])
SelectedFeatures2

array(['31593_at', '31698_at', '31947_r_at', '40029_at', '32098_at',
       '32105_f_at', '33745_at', '33783_at', '35691_r_at', '36923_at',
       '41204_s_at', '32817_at', '34399_at', '34410_at', '33153_at',
       '1486_at', '1462_s_at', '999_at', '328_at', '31913_at', '32463_at',
       '33600_at', '37158_at', '38504_at', '39992_at', '40159_r_at',
       '32204_at', '1687_s_at', '1172_at', '378_s_at', '341_at',
       '335_r_at', '41577_at', '35255_at', '41114_at', '1103_at',
       '32984_s_at', '32935_at', '38661_at', '34831_at', '360_at',
       '41017_at', '40006_at', '39989_at', '41820_s_at', '36170_at',
       '31330_at', '31538_at', '31584_at', '31727_at'], dtype=object)

In [ ]:
'''IG =  mutual_info_classif(X, y, discrete_features=True) #using the mutual_info to calculate information gain of each attribute
for i in range(len(X.columns)):
    if X.columns[i]=='109_at':
      print(X.columns[i],IG[i],ig[i],sep='   ')'''


109_at   0.692201702962276   0.04246274739076111


*Info Gain Ratio Using Library*

In [ ]:
IG =  mutual_info_classif(X, y, discrete_features=True) #using the mutual_info to calculate information gain of each attribute
new_ig=[]
for i in range(len(X.columns)):
      new_ig.append(IG[i]/iv[i])
      


In [184]:
a1 = pd.DataFrame(new_ig)
b1 = pd.DataFrame(X.columns)
info2_ = pd.concat([a1,b1], axis=1)
info2_.columns = ['Score','Features']
info2_

,Score,Features
0,0.253627,31307_at
1,0.379228,31309_r_at
2,0.120911,31313_at
3,0.423969,31314_at
4,0.379228,31315_at
...,...,...
9940,0.302509,105_at
9941,0.693147,106_at
9942,0.510816,107_at
9943,0.215756,108_g_at


In [185]:
top2 = info2_.nlargest(50,'Score')
SelectedFeatures2_= np.array(top2['Features'])
SelectedFeatures2_

array(['38576_at', '38836_at', '33516_at', '39271_at', '32134_at',
       '37539_at', '33605_at', '34640_at', '36703_at', '37161_at',
       '39642_at', '41371_at', '41621_i_at', '31790_at', '32044_at',
       '33292_at', '33785_at', '34198_at', '34269_at', '36065_at',
       '36818_at', '37577_at', '38273_at', '38276_at', '38647_at',
       '38717_at', '39052_at', '39752_at', '40133_s_at', '40145_at',
       '40785_g_at', '40805_at', '40817_at', '40837_at', '41131_f_at',
       '41138_at', '41236_at', '32753_at', '33434_at', '33445_at',
       '33835_at', '33900_at', '33906_at', '33912_at', '34312_at',
       '34363_at', '34857_at', '34886_at', '35285_at', '36120_at'],
      dtype=object)

*SymmetricalUncertainty*

In [ ]:
#entropy
def entropy(Y):
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

#Joint Entropy
def jEntropy(Y,X):
    YX = np.c_[Y,X]
    return entropy(YX)

#conditional entropy
def cEntropy(Y, X):
    return jEntropy(Y, X) - entropy(X)

#SU
def symmetricalUncertain(Y,X):
    n = float(y.shape[0])
    vals = np.unique(Y)
    # Computing Entropy for the feature x. 
    Hx = entropy(X)
    # Computing Entropy for the feature y.
    Hy = entropy(Y)
    #Computing Joint entropy between x and y.
    Hxy = jEntropy(Y,X)
    IG = Hx-Hxy
    return 2*IG/(Hx+Hy)

In [ ]:
top_n1 =[]
for i in X.columns.values:
    X[i]=X[i].astype('float64')
    top_n1.append(symmetricalUncertain(X[i],y))
print (top_n1)
col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n1)
b1 = pd.DataFrame(col_name1)
info3 = pd.concat([a1,b1], axis=1)
info3.columns = ['Score','Features']
info3


[-1.082757413285167, -1.2010731218063744, -0.8347611799093558, -1.2482309393723485, -1.2010731218063744, -1.1861311713396452, -1.2766387742077991, -1.2274814503093183, -1.2766387742077991, -1.2130355338886825, -1.1637319277708345, -1.2825486299016082, -1.2280137598618739, -1.2880038679556594, -1.2970632704138985, -1.234928464352561, -1.1290555853166577, -1.081408861996954, -1.0497701314243342, -1.2880038679556594, -1.2880038679556594, -1.2010731218063744, -1.1290555853166577, -1.2336759298528155, -1.2650659715401062, -1.2650659715401062, -1.2567854542312593, -1.1277950238822443, -1.2563522566783432, -1.1861311713396452, -1.2341167283046541, -1.1277950238822443, -1.2650659715401062, -1.1861311713396452, -1.1107224119393428, -1.1861311713396452, -1.2280137598618739, -1.2130355338886825, -0.7142969768177007, -1.2280137598618739, -1.2766387742077991, -1.2727087104685024, -1.0940900098908064, -1.2567854542312593, -1.1861311713396452, -1.274848474128099, -1.0497701314243342, -1.2567854542312

,Score,Features
0,-1.082757,31307_at
1,-1.201073,31309_r_at
2,-0.834761,31313_at
3,-1.248231,31314_at
4,-1.201073,31315_at
...,...,...
9940,-1.163732,105_at
9941,-1.176019,106_at
9942,-1.256785,107_at
9943,-1.044031,108_g_at


In [ ]:
top3 = info3.nlargest(50,'Score')
SelectedFeatures3 = np.array(top3['Features'])
SelectedFeatures3

array(['31593_at', '31698_at', '31947_r_at', '40029_at', '32098_at',
       '32105_f_at', '33745_at', '33783_at', '35691_r_at', '36923_at',
       '41204_s_at', '32817_at', '34399_at', '34410_at', '33153_at',
       '1486_at', '1462_s_at', '999_at', '328_at', '41577_at', '31913_at',
       '32463_at', '33600_at', '37158_at', '38504_at', '39992_at',
       '40159_r_at', '32204_at', '1687_s_at', '1172_at', '378_s_at',
       '341_at', '335_r_at', '31769_at', '35118_at', '35509_at',
       '34485_r_at', '41114_at', '31825_at', '34241_at', '36541_at',
       '38976_at', '39359_s_at', '40809_at', '39177_r_at', '41521_at',
       '2049_s_at', '1597_at', '1504_s_at', '1210_s_at'], dtype=object)

#Selecting 5k Subset

In [ ]:
def top_fs_union(n):
    f1 = SelectedFeatures1[:n]
    f2 = SelectedFeatures2[:n]
    f3 = SelectedFeatures3[:n]
    f_3k = reduce(np.union1d, (f1,f2,f3))
    f_2k = ((len(f_3k)*2)//3)
    f_2k_random = np.random.choice(f_3k,f_2k,replace=False)
    f_5k = reduce(np.union1d, (f_3k,f_2k_random))
    #print (f_5k)
    return(f_5k)

In [ ]:
'''f1 = SelectedFeatures1[:5]
f2 = SelectedFeatures2[:5]
f3 = SelectedFeatures3[:5]
f_3k = reduce(np.union1d, (f1,f2,f3))
f_2k = ((len(f_3k)*2)//3)
f_2k_random = np.random.choice(f_3k,f_2k,replace=False)
f_5k = reduce(np.union1d, (f_3k,f_2k_random))
f_5k'''

#Assemble Function RUN

In [ ]:
def assemble_run(n,classifier,x_train, x_test, y_train, y_test):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['class'], axis=0)
  df = data.loc[:,top_fs]
  X = df.drop(['class'], axis=1)
  y = df['class']
  model =classifier()#modify the code ,keep classifier in different function and call here
  model.fit(x_train, y_train)
  print (model.score(x_test,y_test))

#Classifier run with feature selection




In [181]:
#SVM
for i in (5,10,15,20,25,30,40,50):
    print('Score for SVM by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,SVC,x_train, x_test, y_train, y_test)
print('\n\n')

#NaiveBayes
for i in (5,10,15,20,25,30,40,50):
    print('Score for Naive Bayes by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,GaussianNB,x_train, x_test, y_train, y_test)
print('\n\n')

#DecisionTree
for i in (5,10,15,20,25,30,40,50):
    print('Score for DecisionTree by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,DecisionTreeClassifier,x_train, x_test, y_train, y_test)
print('\n\n')

#KNN
for i in (5,10,15,20,25,30,40,50):
    print('Score for KNN by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,KNeighborsClassifier,x_train, x_test, y_train, y_test)
print('\n\n')


Score for SVM by Union of 5 Best Features: 0.42857142857142855
Score for SVM by Union of 10 Best Features: 0.42857142857142855
Score for SVM by Union of 15 Best Features: 0.42857142857142855
Score for SVM by Union of 20 Best Features: 0.42857142857142855
Score for SVM by Union of 25 Best Features: 0.42857142857142855
Score for SVM by Union of 30 Best Features: 0.42857142857142855
Score for SVM by Union of 40 Best Features: 0.42857142857142855
Score for SVM by Union of 50 Best Features: 0.42857142857142855



Score for Naive Bayes by Union of 5 Best Features: 0.5
Score for Naive Bayes by Union of 10 Best Features: 0.5
Score for Naive Bayes by Union of 15 Best Features: 0.5
Score for Naive Bayes by Union of 20 Best Features: 0.5
Score for Naive Bayes by Union of 25 Best Features: 0.5
Score for Naive Bayes by Union of 30 Best Features: 0.5
Score for Naive Bayes by Union of 40 Best Features: 0.5
Score for Naive Bayes by Union of 50 Best Features: 0.5



Score for DecisionTree by Union of 5

#Classifier run without feature selection


In [182]:
model1=SVC()
model2=GaussianNB()
model3 = DecisionTreeClassifier()
model4 = KNeighborsClassifier()
model1.fit(x_train, y_train)
print ('SVC Score:' ,model1.score(x_test, y_test))
model2.fit(x_train, y_train)
print ('NB Score:' ,model2.score(x_test, y_test))
model3.fit(x_train, y_train)
print ('DT Score:' ,model3.score(x_test, y_test))
model4.fit(x_train, y_train)
print ('KNN Score:' ,model4.score(x_test, y_test))

SVC Score: 0.42857142857142855
NB Score: 0.5
DT Score: 0.7142857142857143
KNN Score: 0.42857142857142855
